# Google Work Hours
Extracts work hours based on location data.
Registers the earliest and latest entry per day for a specific geospatial location, the work place.
Based on this it calculates the length of the work day.

In [ ]:
# -*- coding: utf-8 -*-
from folium import Map, Marker, Icon, Circle
from pyproj import Proj, transform
from datetime import datetime, timezone
from os.path import exists, isfile
from distutils.command.config import config

import json
import math
import pandas as pd
import matplotlib.pyplot as plt

from latlon_to_utm import GPStoUTM

def coordConv(Lat, Long):
    lat = float(int(Lat)/(10**7))
    long = float(int(Long)/(10**7))
    return [lat, long]


configuration = {}
with open("config.json", "r", encoding="utf-8") as config_file_handle:
    configuration = json.loads( config_file_handle.read().strip() )


if configuration:
    #Starting timestamp to extract
    origoTs = datetime.strptime( configuration["date_start"], '%Y%m%d' ).replace( tzinfo=timezone.utc )
    
    #Set origo for geo-fencing
    gps = GPStoUTM()
    #Retrieved from google maps (WSG84 coordinate system)
    CoordWork = [configuration["latitude"], configuration["longitude"]]
    CoordWorkM = gps.LLtoUTM(configuration["latitude"], configuration["longitude"])
    LocationDataFile = None

## Import location data file
Location data is regularly altered.  Two version of timestamp information is parsed currently and three fields are used, timestamp, latitude and longtitude.

In [ ]:
LocationDataFile = input( "Data file: " ).replace('"', '')
if LocationDataFile and exists( LocationDataFile ):
    JsonData = {}
    with open(LocationDataFile, "r", encoding="utf-8") as F:
        JsonData = json.loads( F.read().strip() )["locations"]
    df = pd.DataFrame( JsonData )

## Load data
loads the data and parses its location info into dasets per day.

In [ ]:
#Plot
source_columns = df.columns
timestampColumn = "timestampMs" if "timestampMs" in source_columns else "timestamp"
cols = ["latitudeE7","longitudeE7", timestampColumn]
if timestampColumn == "timestamp":
    df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%dT%H:%M:%S.%f' )
    
EuclidHist = []
dailySets  = {}
preKey     = 0
for index, row in df[cols].iterrows():
    if timestampColumn == "timestampMs":
        row["timestampMs"] = datetime.fromtimestamp( int(row.timestampMs)/1000.0 )
    
    if row[timestampColumn] < origoTs:
        continue
        
    ts = row[timestampColumn]
    key = int( ts.strftime("%Y%m%d") )
    if key != preKey and key not in dailySets:
        dailySets.update({key:[]})
            
    tmpCoord  = coordConv(row.latitudeE7, row.longitudeE7)
    tmpCoordM = gps.LLtoUTM(tmpCoord[0], tmpCoord[1])
    
    eu = math.dist( CoordWorkM, tmpCoordM )
    EuclidHist.append(eu)
    
    dailySets[key].append({
        "lat":       tmpCoord[0],
        "long":      tmpCoord[1],
        "euclid":    eu,
        "timestamp": ts
    })

## Extract euclid distance of < n

In [ ]:
dfHist = pd.DataFrame( EuclidHist )
dfHist[dfHist[0] < configuration["max_euclid"] ].hist(bins=50, figsize=(20,10))

## Show geospatial info on map

In [ ]:
euclid_distance = configuration["euclid_distance"]
workHours = {}
popuptxt = "{}\n[{}:{}]"
origo_ts_int = int( configuration["date_start"] )

my_map = Map(location=CoordWork,
            zoom_start=configuration["zoom_start"])

Circle( location=CoordWork, 
        radius=euclid_distance,
        popup='Work Area',
        line_color='black',
        fill_color='green',
        fill_opacity=0.25).add_to(my_map)

for day in dailySets:
    if day < origo_ts_int:
        continue
    tsFirst = None
    tsLast = None
    for event in dailySets[day]:
        if event["euclid"] <= euclid_distance:      #Within geofencing radius
            if not tsFirst:
                tsFirst = event
                tsLast = event
            else:
                if tsLast["timestamp"] < event["timestamp"]:
                    tsLast = event
    if tsFirst and tsLast:
        Marker( location=[tsFirst["lat"],tsFirst["long"]], icon=Icon(color="green"),
                popup=popuptxt.format(tsFirst["timestamp"], tsFirst["lat"],tsFirst["long"] ) ).add_to(my_map)

        Marker( location=[tsLast["lat"],tsLast["long"]], icon=Icon(color="red"),
                popup=popuptxt.format(tsLast["timestamp"], tsLast["lat"],tsLast["long"] ) ).add_to(my_map)
    workHours.update({day: [tsFirst,tsLast]})

my_map

## Generate excel

In [ ]:
hours = []
day_min = None
day_max = None
for day in workHours:
    date = datetime.strptime( str(day), "%Y%m%d" )
    if day_min == None or day_min > date:
        day_min = date
    if day_max == None or date > day_max:
        day_max = date
        
    weekend = True if date.weekday() >= 5 else False
        
    if workHours[day][0] is None or workHours[day][1] is None:
        minHours = None
        hours.append({"date":date.strftime("%Y%m%d"), 
                      "start":None, 
                      "end":None, 
                      "minutes":0.0, 
                      "weekend":weekend, 
                      "7.5h":None})
    else:
        start = workHours[day][0]["timestamp"]
        end   = workHours[day][1]["timestamp"]
        minHours = True if (((end-start).seconds/3600.0) > 7.5) else False
        hours.append({"date":date.strftime("%Y%m%d"), 
                      "start":start.strftime("%Y%m%d_%H%M"), 
                      "end":end.strftime("%Y%m%d_%H%M"), 
                      "minutes":(end-start).seconds/60.0, 
                      "weekend":weekend, 
                      "7.5h":minHours})

        
dfHours = pd.DataFrame(hours)
dfHours.to_excel("workHours-{}-{}.xlsx".format(datetime.strftime( day_min, "%Y%m%d" ),
                                                datetime.strftime( day_max, "%Y%m%d" )),
                                                 sheet_name="Work Hours", header=True, index=False)
dfHours.head()

## Visualization

In [ ]:
dfA = dfHours.set_index("date")
plt.style.use("fivethirtyeight")
plt.figure(figsize=(50, 20))
plt.xlabel("date")
plt.ylabel("minutes")
plt.title("Sample Time Series Plot")
plt.plot(dfA["minutes"]/60)
